### Surface adsorption
This notebook allows you to position an adsorbate on a surface.
You will need:
- the input for a surface calculation (SLAB)
- the structure of the adsorbate. This can be read from file, a single atom or a nanoparticle
You will be able to move and rotate the structure. For this step, it's recommended to start with no displacement, visualise the structure and try different shifts and rotations until the desired one is achieved. The last cell will display the structure.

In [1]:
import sys
sys.path.insert(1, '../../functions')
from os.path import join
import os
import numpy as np

from ase.cluster import wulff_construction
from ase.visualize import view
from ase import Atoms


from crystal_io import read_input
from crystal_io import write_input

from settings import runcry
from settings import vesta
from geom_modification import make_supercell
from geom_modification import insert_atom
from geom_modification import displace_atom
from visualisation_tools import gui2cif

from ase.io.crystal import read_crystal

In [2]:
#Variables used in the whole notebook
directory = '../data/adsorption' # directory where the original input is saved. This must be a SLAB input
input_name = 'BN.d12' # name of the original input
final_input_name = 'BN_5x5_NP.d12'
file_path = join(directory,input_name)
final_file_path = join(directory,final_input_name)

#Supercell generation (create a supercell of the surface)
supercell_generation = True
expansion_matrix = [5, 0, 0, 5] 

#Testgeom
testgeom = True

#Run the calculation
run = True

### Choose the structure to position on the surface
Select only one of the following three options.

#### Read a structure from file
ASE can read a large range of file formats, see this <a href="https://wiki.fysik.dtu.dk/ase/ase/io/io.html">page</a> for more info.

In [3]:
atoms = 'Insert here the file to be read'
view(atoms, viewer='x3d')

AttributeError: 'str' object has no attribute 'write'

#### Single atom

In [3]:
atoms = Atoms('O')

#### Nanoparticle
Generate a nanoparticle using ASE. See this page <a href="https://wiki.fysik.dtu.dk/ase/ase/cluster/cluster.html">page</a> for more info.

In [5]:
# Create the NP by using ASE

#Atom type
atom_type = 'Cu'

#Type of lattice
lattice= 'fcc'
    
#Miller indeces of the surfaces
surfaces = [(1, 0, 0), (1, 1, 0), (1, 1, 1)]

#Surface energies
esurf = [1.0, 1.1, 0.9]   

#Lattice constant
lc = 3.61000

# Number of atoms (ASE will generate a NP as close as possible to the size requested)
size =5  
atoms = wulff_construction(atom_type, surfaces, esurf,
                           size, lattice,
                           rounding='above', latticeconstant=lc)
n_atoms = len(atoms.get_atomic_numbers())
disp = False
view(atoms, viewer='x3d')

### Move and rotate the structure
This step will allow you to position the adsorbate on the exact surface coordinates.

In [6]:
# Move and rotate the structure
atoms.rotate(45,'x')
atoms.rotate(35,'y')

#Index of the first atom to be displaced (check the label of the first atom belonging to the adsorbate)
first_disp = 51
#Displacement vector (this will normally require some trial and error)
disp = np.array([4.372597334022, 2.51879531, 4.45])

In [10]:
#Read the input (SLAB)
geom_block,optgeom_block,bs_block,func_block,scf_block = read_input(file_path)

if supercell_generation == True:
    geom_block,scf_block = make_supercell(geom_block,scf_block,expansion_matrix)

geom_block = insert_atom(geom_block,atoms.get_atomic_numbers().tolist(),atoms.get_positions().tolist()) 

if type(disp) != bool:
    disp_atoms = [i for i in range(first_disp,(first_disp+n_atoms))]
    disp_coord = np.tile(disp, (n_atoms, 1)).tolist()
    geom_block = displace_atom(geom_block,disp_atoms,disp_coord) 
    
#Insert EXTPRT and TESTGEOM if not present in the input
if testgeom == True:
    if 'EXTPRT\n' not in geom_block:
        geom_block.insert(len(geom_block)-1,'EXTPRT\n')
    if 'TESTGEOM\n' not in geom_block:
        geom_block.insert(len(geom_block)-1,'TESTGEOM\n')

write_input(final_file_path,geom_block,bs_block,func_block,scf_block,optgeom_block) 

if run == True:
    runcry(final_file_path[:-4])
    #Visualise the structure
    gui2cif(final_file_path[:-4]+'.gui')
    cif_file_name = final_file_path[:-4]+'.cif'
    vesta(cif_file_name)    

In [11]:
view(read_crystal(final_file_path[:-4]+'.gui'),viewer='x3d')